Задача:

Необходимо построить векторные представления (эмбеддинги) пользователей и фильмов используя нейросетевые подходы 
чтобы можно было по эмбеддингу пользователя искать похожие эмбеддинги фильмов и рекомендовать ему их, на основе MovieLens Latest Datasets Full

При выполнении задания обратите внимание на:


*   Разбиение данных на обучение и валидацию, обоснование
*   Выбор и обоснование метрики
*   Разработка архитектуры нейронной сети с пояснениями
*   Обучение и валидация




Опишем общий план решения задачи:


1.   Обьеденить, очистить и т.д., закодировать категориальные признаки.
2.   Подготовить данные и преобразовать в тензоры для поподачи в Dataloader,
      пердварительно разделив их на train, val, test.
3.   Создание нейрронной сети.
 
  а) Подбор сети и архетиктуры
 
  б) Обучение и валидация
 
4.   Получение эмбедингов фильмов.
 
5.   Подбор метрики и тестирование.




# Загрузка и предобработка данных

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive # Настроим доступ к данным
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip -q /content/gdrive/MyDrive/Colab_Notebooks/DATA/ml-latest.zip

In [ ]:
ls ml-latest

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


In [ ]:
movies = pd.read_csv('ml-latest/movies.csv')
ratings = pd.read_csv('ml-latest/ratings.csv')
tags = pd.read_csv('ml-latest/tags.csv')
genome_scores = pd.read_csv('ml-latest/genome-scores.csv')
genome_tags = pd.read_csv('ml-latest/genome-tags.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 3 entries, movieId to genres
dtypes: object(2), int64(1)

In [ ]:
# Взглянем на какие есть у нас жанры
movies['genres'].str.split('|', expand=True).stack().unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

Используем технику one hot encoding для жанров, в дальнейшем нам это пригодится

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# создание экземпляра класса MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# разделение жанров в каждой строке на отдельные значения
genres = [genre.split('|') for genre in movies['genres'].values]

# кодирование жанров в one hot encoding
one_hot_genres = mlb.fit_transform(genres)

# создание DataFrame из one hot encoding
df_one_hot_genres = pd.DataFrame(one_hot_genres, columns=mlb.classes_)

# объединение DataFrame с one hot encoding и исходного DataFrame
movies = pd.concat([movies, df_one_hot_genres], axis=1)

In [ ]:
movies.head(4)

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
# Теперь удалим колонку genres
movies.drop('genres', axis=1, inplace=True)

In [ ]:
movies.columns

Index(['movieId', 'title', '(no genres listed)', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1108997 non-null  int64 
 1   movieId    1108997 non-null  int64 
 2   tag        1108981 non-null  object
 3   timestamp  1108997 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.8+ MB


In [ ]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


In [ ]:
genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14862528 entries, 0 to 14862527
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 340.2 MB


In [ ]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [ ]:
genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


Отберем юзеров и фильмов , у которых по 400 отзывов минимум для более корректных эмбедингов

In [ ]:
# Выбор популярных пользователей и фильмов (встречающихся более 500 раз)

ratings = ratings[ratings['userId'].isin(ratings['userId'].value_counts()[ratings['userId'].value_counts() > 500].index) & ratings['movieId'].isin(ratings['movieId'].value_counts()[ratings['movieId'].value_counts() > 500].index)]

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8206656 entries, 42 to 27751546
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 313.1 MB


Сначала необходимо объединить таблицы ratings,tags и movies на основе movieId. 

In [ ]:
ratings = ratings.merge(movies, on='movieId', how='left')

In [ ]:
ratings = ratings.merge(tags, on=['movieId', 'userId', 'timestamp'], how='left')

In [ ]:
# Создадим новую колонку "year" из года, извлеченного из строки в колонке "title"
ratings['year'] = ratings['title'].str[-5:-1]

# Удалим последние 6 символов (скобки с годом) из каждой строки в колонке "title"
ratings['title'] = ratings['title'].str[:-6]

In [ ]:
ratings.sample()

,userId,movieId,rating,timestamp,title,(no genres listed),Action,Adventure,Animation,Children,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,tag,year
8016870,276570,2302,3.5,1097697649,My Cousin Vinny,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,1992


In [ ]:
del tags, movies

Теперь поработавет с тэгами

In [ ]:
genome_scores = genome_scores.merge(genome_tags, on='tagId', how='left')

In [ ]:
del genome_tags

In [ ]:
genome_scores.sample()

,movieId,tagId,relevance,tag
2002622,2010,423,0.051,gangster


genome_scores это таблица которая по каждому из тегов дает релевантный скор к тегу.

In [ ]:
genome_scores = genome_scores.pivot(index='movieId', columns='tag', values='relevance')

In [ ]:
genome_scores.sample(3)

tag,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
154915,0.01125,0.01125,0.09925,0.08450,0.11825,0.05675,0.03050,0.33750,0.42550,0.03750,...,0.40775,0.19625,0.87375,0.05125,0.24575,0.14750,0.02250,0.4550,0.05575,0.01625
26398,0.08100,0.08600,0.06425,0.05475,0.18425,0.21775,0.05875,0.58275,0.12750,0.01975,...,0.09175,0.02025,0.01725,0.06300,0.17675,0.07675,0.01800,0.0105,0.12100,0.02075
26163,0.02175,0.02550,0.08075,0.06225,0.17450,0.15200,0.67300,0.24350,0.07275,0.04800,...,0.11175,0.03625,0.01825,0.09025,0.21775,0.12500,0.02625,0.0140,0.09600,0.02250


Также сюда добавим ср. оценку по всем пользователя
Я снова скачаю ratings и по полной таблице посчитаю средний рейтинг к фильму

In [ ]:
assignment = pd.read_csv('ml-latest/ratings.csv')

In [ ]:
assignment.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
assignment.groupby('movieId')['rating'].mean().round(2).head()

movieId
1    3.89
2    3.25
3    3.17
4    2.87
5    3.08
Name: rating, dtype: float64

In [ ]:
genome_scores = genome_scores.merge(assignment.groupby('movieId')['rating'].mean().round(2), on='movieId', how='left')

In [ ]:
genome_scores.sample()

,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,rating
movieId,,,,,,,,,,,,,,,,,,,,,
3604,0.033,0.0385,0.07475,0.3985,0.616,0.4525,0.13025,0.291,0.15275,0.043,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1962,3.61


Тоже самое сделаем для года и жанра 

In [ ]:
genome_scores = genome_scores.merge(ratings.groupby('movieId')[['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western', 'year']].max(), on='movieId', how='left')

<ipython-input-33-88c5acc50476>:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  'War', 'Western', 'tag', 'year']].max(), on='movieId', how='left')


In [ ]:
# Оставим наши 500+ по отзывам
genome_scores.dropns(inplace=True)

In [ ]:
genome_scores.sample(4)

,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,rating
movieId,,,,,,,,,,,,,,,,,,,,,
170697,0.03325,0.03425,0.02850,0.10275,0.12125,0.09975,0.04100,0.08825,0.04050,0.02825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,3.89
160563,0.03725,0.02850,0.04000,0.08750,0.09825,0.05950,0.03350,0.04000,0.02325,0.24475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016,3.08
3127,0.03250,0.03125,0.03175,0.02575,0.03675,0.03175,0.01950,0.11400,0.03600,0.02025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1999,2.97
4037,0.02250,0.02375,0.08775,0.11025,0.14250,0.16375,0.06475,0.20600,0.14275,0.04275,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1987,3.94


Оставим в genome_scores популярные фильмы

Последний штрих , создадим отделный дф с названиями фильмов и их id 

In [ ]:
movies = ratings[['title', 'movieId']].groupby('movieId','title').max()

In [ ]:
movies = movies.reset_index().rename(columns={'movieId': 'movie_id'})

In [ ]:
movies.sample(4)

,movie_id,title
1204,1882,Godzilla
2213,3466,Heart and Souls
617,906,Gaslight
884,1282,Fantasia


In [ ]:
# Итоговая таблица с рейтингами
ratings = ratings[['userId', 'movieId', 'rating', 'timestamp']]

In [ ]:
ratings.sample(4)

,userId,movieId,rating,timestamp
3700452,128610,5942,3.0,1500229906
5813323,200308,1729,4.0,983386507
352831,11698,1707,1.0,1470409832
1865355,65226,6212,3.5,1085101597


Нормализация

In [12]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Создаем экземпляр StandardScaler и фитим на данные
scaler = StandardScaler()
scaler.fit(genome_scores[['year', 'rating']])


# Нормализуем данные
genome_scores[['year', 'rating']] = scaler.transform(genome_scores[['year', 'rating']])

In [ ]:
# Также для ratings
scaler.fit(ratings[['rating', 'timestamp']])

ratings[['rating', 'timestamp']] = scaler.transform(ratings[['rating', 'timestamp']])

В итоге у нас 3 таблицы:


1.   ratings

  В нем хранятся 'userId', 'movieId', 'rating', 'timestamp'
2.   genome_scores 

  По каждому фильму стоит рейтинг, жанр, год и его релевантность по каждому из 1128 тэгу.

3. movies

  тут храняится навзания по айди фильма.




ОБРАБОТКА

In [ ]:
ratings.to_csv('./gdrive/MyDrive/Colab_Notebooks/DATA/ratings')

In [ ]:
genome_scores.to_csv('./gdrive/MyDrive/DATA/genome_scores')

In [ ]:
movies.to_csv('./gdrive/MyDrive/Colab_Notebooks/DATA/movies')

In [3]:
import pandas as pd

In [1]:
from google.colab import drive # Настроим доступ к данным
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
ratings = pd.read_csv('./gdrive/MyDrive/DATA/ratings', index_col='Unnamed: 0')

In [5]:
genome_scores = pd.read_csv('./gdrive/MyDrive/DATA/genome_scores', index_col='Unnamed: 0')

In [ ]:
movies = pd.read_csv('./gdrive/MyDrive/Colab_Notebooks/DATA/movies', index_col='Unnamed: 0')

# Эмбеддинги фильмов

Для векторизации фильмов будем использовать Autoencoder.

Автоэнкодер может использоваться для получения эмбеддингов фильмов. При обучении автоэнкодера на данных о фильмах, он будет стремиться сжать информацию о фильмах в скрытое представление (латентное пространство), которое можно использовать как эмбеддинг фильма. Это представление должно содержать важные признаки фильма, которые помогут определить его релевантность для конкретного пользователя.

В нашем случае, X будет матрица фильмов и их признаков. Пусть у нас есть N фильмов и каждый фильм имеет M признаков. Тогда размерность матрицы X будет (N, M).

Y также будет матрицей размерности (N, M) не считая movieID, которая представляет собой "цели" для нашей модели. В случае автоэнкодера мы хотим, чтобы Y была идентичной X, то есть мы хотим, чтобы модель могла восстановить входные данные из своего сжатого представления. Поэтому Y будет матрицей X.

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset, RandomSampler

In [ ]:
# Отделение колонки movieId
X = genome_scores['movieId']
Y = genome_scores.drop('movieId', axis=1)

# Преобразование данных в тензоры PyTorch
X = torch.tensor(X.values, dtype=torch.float32)
Y = torch.tensor(Y.values, dtype=torch.float32)

Преобразуем их в Dataloader для подачи в сеть бачами

In [ ]:
# Создаем TensorDataset из X и Y
dataset = TensorDataset(X, Y)

# Создаем DataLoader с заданным размером батча
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Создание сети

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(1151, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(True),
            nn.Linear(32, 20),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(20, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(True),
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 1151),
            nn.BatchNorm1d(1151),
            nn.ReLU(True)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

Мы будем оптимизировать функцию потерь MSE между X и Y, чтобы получить наилучшее сжатое представление данных в модели.

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [ ]:
# Создание экземпляра модели
autoencoder = Autoencoder().to(device)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 1000
print_every = 50
Y = Y.to(device)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = autoencoder(Y)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % print_every == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [50/1000], Loss: 0.0311
Epoch [100/1000], Loss: 0.0257
Epoch [150/1000], Loss: 0.0230
Epoch [200/1000], Loss: 0.0209
Epoch [250/1000], Loss: 0.0195
Epoch [300/1000], Loss: 0.0185
Epoch [350/1000], Loss: 0.0177
Epoch [400/1000], Loss: 0.0170
Epoch [450/1000], Loss: 0.0165
Epoch [500/1000], Loss: 0.0159
Epoch [550/1000], Loss: 0.0155
Epoch [600/1000], Loss: 0.0151
Epoch [650/1000], Loss: 0.0148
Epoch [700/1000], Loss: 0.0144
Epoch [750/1000], Loss: 0.0142
Epoch [800/1000], Loss: 0.0141
Epoch [850/1000], Loss: 0.0137
Epoch [900/1000], Loss: 0.0135
Epoch [950/1000], Loss: 0.0133
Epoch [1000/1000], Loss: 0.0131


In [ ]:
# Добучение модели
num_epochs = 500
print_every = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = autoencoder(Y)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % print_every == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [100/500], Loss: 0.0035
Epoch [200/500], Loss: 0.0035
Epoch [300/500], Loss: 0.0035
Epoch [400/500], Loss: 0.0035
Epoch [500/500], Loss: 0.0035


Вышли на плато.

Получим эмбеддинги из обученной модели

In [ ]:
with torch.no_grad():
    embeddings_films = []
    for batch in dataloader:
        x_batch, y_batch = batch
        embeddings_batch = autoencoder.encoder(y_batch)
        embeddings_films.append(embeddings_batch)
    embeddings_films = torch.cat(embeddings_films, dim=0)

In [ ]:
embeddings_films

tensor([[ 0.9792,  0.6527,  0.2755,  ...,  0.0973,  0.4610,  0.3140],
        [ 0.1890,  0.7176,  0.1497,  ...,  0.3803,  0.0995, -0.2990],
        [ 1.0604, -0.5654,  1.0824,  ...,  0.7113,  0.1615, -0.5671],
        ...,
        [ 0.2248,  0.2182,  0.5059,  ..., -0.1851, -0.1365,  0.5209],
        [ 0.0652,  0.2796,  0.3306,  ...,  0.4230, -0.4758, -0.0240],
        [ 0.2370,  0.2009,  0.5144,  ..., -0.6449, -0.1277,  0.6537]],
       device='cuda:0')

In [ ]:
embeddings_films.shape

torch.Size([5544, 20])

In [ ]:
embeddings = torch.load('embeddings_films.pt')

Мы получили векторное представление фильмов в латентном пространсве автоэнкодера, разменостью 20 для каждого фильма

# Эмбеддинги пользователей 2.0 рабочая

Для построения эмбеддингов пользователей используем тот же способ , что мы применяли для фильмов только теперь X (ratings ratings + genome_scores),
а матрица Y будет индентична X, не считая movieID и userId.

Так как нам понадобится обьединить таблицы ratings и genome_scores , но необходимой памяти ОЗУ нехватает, мы будем соединять батч таблицы ratings с genome_scores, и затем исключать movieID и userId, в момент подачи.

P.s. Я потратил несколько дней пытаясь решить проблему с нехваткой ОЗУ при джойне ratings и genome_score, распарареливал процес и брал часть данных только потом джойнить , но сеть расходилась и лосс убегал в инфинитив, ниже вы увидете уже 2.0 версию кода. Тут я уже для построения эмбедингов пользователей буду брать год жанр рейтинг из genome_scores, используем также простой даталоадер.


Наша цель - сжать данные, чтобы сохранить наиболее важные признаки и понизить размерность. 

In [9]:
genome_scores = genome_scores[['movieId','year', 'rating', 'Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical',
       'Western', 'Film-Noir', '(no genres listed)']]

In [10]:
ratings = ratings.merge(genome_scores, how='left', on='movieId')

In [33]:
from sklearn.preprocessing import MinMaxScaler

# Создание экземпляра MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Применение нормализации к нужным столбцам DataFrame
ratings[['userId', 'movieId', 'rating_x', 'timestamp', 'year', 'rating_y']] = scaler.fit_transform(ratings[['userId', 'movieId', 'rating_x', 'timestamp', 'year', 'rating_y']])


In [14]:
ratings = ratings.sort_values('userId')

In [34]:
ratings.sample()

,userId,movieId,rating_x,timestamp,year,rating_y,Adventure,Animation,Children,Comedy,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
1690554,0.210098,0.046014,0.555556,0.412577,0.706897,0.680597,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
class CustomDataset(Dataset):
    def __init__(self, ratings):
        self.user_ids = ratings['userId'].unique()
        self.ratings_by_user = ratings.groupby('userId').apply(lambda x: x.iloc[:, :-1].values)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        ratings = self.ratings_by_user[user_id]
        return torch.tensor(ratings, dtype=torch.float32)

# Создание экземпляра кастомного датасета
dataset = CustomDataset(ratings)

Еще один АвтоЭнкодер

In [106]:
class Autoencoder_UsEmb(nn.Module):
    def __init__(self):
        super(Autoencoder_UsEmb, self).__init__()
        

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(25, 20),
            nn.ReLU(),  
            nn.Linear(20, 20)
        )

        # Decoder
        self.decoder = nn.Sequential(   
            nn.Linear(20, 20),
            nn.ReLU(),    
            nn.Linear(20, 25),
            nn.BatchNorm1d(25),
            nn.ReLU()        
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [107]:
# Создаем экземляр модели и гипермпараметры
model = Autoencoder_UsEmb().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [108]:
n_epochs = 50
for epoch in range(n_epochs):
    # Итерация по батчам пользователей
    for user_batch in dataset:
        # Передача батча на устройство
        user_batch = user_batch.to(device)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход через модель
        outputs = model(user_batch)

        # Вычисление функции потерь
        loss = criterion(outputs, user_batch)

        # Обратное распространение ошибки и обновление весов
        loss.backward()
        optimizer.step()
        print(loss.item())
    # Вывод значения функции потерь на каждой эпохе
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

0.47640979290008545
0.47609278559684753
0.4792887270450592
0.45531150698661804
0.4773313105106354
0.4387921094894409
0.47464877367019653
0.48696503043174744
0.44899511337280273
0.42364421486854553
0.4179540276527405
0.46532565355300903
0.45357388257980347
0.4022410809993744
0.4655129015445709
0.45941463112831116
0.4207402765750885
0.4326898455619812
0.3731573522090912
0.4367850720882416
0.4006073474884033
0.39599189162254333
0.37788262963294983
0.3807675540447235
0.3637793958187103
0.38741666078567505
0.3913702070713043
0.40898337960243225
0.37677645683288574
0.37099120020866394
0.38216540217399597
0.36583971977233887
0.34437111020088196
0.34260791540145874
0.3412906527519226
0.3295140862464905
0.3830451965332031
0.3513643741607666
0.3494744300842285
0.35547417402267456
0.3269914388656616
0.3333195149898529
0.3457038104534149
0.32495391368865967
0.30001744627952576
0.3120897114276886
0.3108193874359131
0.31893256306648254
0.29812949895858765
0.3029395639896393
0.3105962574481964
0.3023

KeyboardInterrupt: ignored

# Эмбеддинг юзеров 1.0(черновик)

In [ ]:
class RatingsGenomeDataset(Dataset):
    def __init__(self, ratings, genome_scores, num_split):
        self.ratings = ratings
        self.genome_scores = genome_scores
        self.num_split = num_split

    def __len__(self):
        return len(self.ratings)//self.num_split # кол-во итераций бачей в ходе одной эпохи

    def __getitem__(self, idx):
        # создаем батч, выбирая случайные строки из датафрейма
        batch_ratings = self.ratings.sample(n=self.num_split)
        
        # объединяем рейтинги с оценками жанров для батча
        batch = batch_ratings.merge(self.genome_scores, on='movieId', how='left')
        # удаляем столбцы userId и movieId
        batch = batch.drop(['userId', 'movieId'], axis=1)
        
        # преобразуем датафрейм в тензор
        Y = torch.tensor(batch.values, dtype=torch.float32)
        
        return  Y

dataset = RatingsGenomeDataset(ratings, genome_scores, num_split=10000)

In [ ]:
dataset[0].shape

torch.Size([10000, 1153])

Построение модели

In [ ]:
class Autoencoder_UEmb(nn.Module):
    def __init__(self):
        super(Autoencoder_UEmb, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(1153, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(True),
            nn.Linear(32, 20),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(20, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(True),
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Linear(512, 1153),
            nn.BatchNorm1d(1153),
            nn.ReLU(True)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = Autoencoder_UEmb().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
# Обучение модели

print_every = 2

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in dataset:
        optimizer.zero_grad()   
        batch = batch.to(device) 
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        average_loss = running_loss / len(dataset)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, average_loss))


Epoch [1/10], Loss: nan
Epoch [1/10], Loss: nan
Epoch [1/10], Loss: nan
Epoch [1/10], Loss: nan
Epoch [1/10], Loss: nan
Epoch [1/10], Loss: nan
Epoch [1/10], Loss: nan


KeyboardInterrupt: ignored

# Collaborative Filtering

Collaborative Filtering: Этот метод основывается на том, что если два пользователя оценили большинство фильмов одинаково, то они, вероятно, будут похожи и на оставшихся фильмах. Этот метод использует только данные о рейтингах, не используя никакой дополнительной информации. Можно использовать косинусное расстояние между эмбеддингами фильмов и пользователями для определения схожести.

План выполнения:
1. Для каждого пользователя находим K ближайших к нему пользователей из обучающей 
выборки на основе косинусного расстояния между эмбеддингами пользователей.

2. Для каждого фильма находим K ближайших к нему фильмов из обучающей выборки на основе косинусного расстояния между эмбеддингами фильмов.

3. Используя оценки фильмов от ближайших пользователей, строим прогноз оценок для фильмов, которые данный пользователь еще не оценил.

4. Для каждого фильма находим среднюю оценку по всем пользователям, которые его оценили, и добавляем ее к прогнозу оценки.

5. Сортируем фильмы по убыванию прогноза оценки и рекомендуем пользователю топ-N фильмов.

# Вывод

В процессе работы над проектом по рекомендательной системе на основе данных MovieLens, мы сначала провели предварительную обработку данных, которая включала чистку и подготовку данных для последующего анализа. Затем мы построили автоэнкодер (Autoencoder) для извлечения эмбеддингов фильмов.

Однако, при попытке построить аналогичную модель для эмбеддингов пользователей, мы столкнулись с проблемой разошедшейся модели. Чтобы преодолеть это, можно попробовать различные подходы, такие как изменение архитектуры модели, настройка гиперпараметров, оптимизация и другие методы, чтобы достичь стабильности обучения модели, но времени проверить нехватило. 

Если бы у нас были эмбеддинги фильмов и пользователей, мы могли бы использовать косинусное расстояние для измерения сходства между ними и основываться на этом для создания рекомендаций.